In [ ]:
%pip install peft transformers accelerate bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 29.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 110.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 88.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 56.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 40.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 18.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 99.8 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstal

In [ ]:
%pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 31.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 18.8 MB/s eta 0:00:00


In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from peft import LoraConfig, get_peft_model

In [95]:
bnb_config = BitsAndBytesConfig(
    load_in_8bit=True,
    llm_int8_threshold=6.0
)

tokenizer = AutoTokenizer.from_pretrained("klyang/MentaLLaMA-chat-7B", use_fast=False)
model = AutoModelForCausalLM.from_pretrained(
    "klyang/MentaLLaMA-chat-7B",
    quantization_config=bnb_config,
    device_map="auto"
)
model.gradient_checkpointing_enable()

In [96]:
lora_config = LoraConfig(
    r=4,
    lora_alpha=16,
    lora_dropout=0.05,
    target_modules=["q_proj", "v_proj"],
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, lora_config)


trainable params: 2,097,152 || all params: 6,740,512,768 || trainable%: 0.0311


In [102]:
import pandas as pd
from datasets import Dataset

# Load CSV using pandas
df = pd.read_csv("/content/DR_generated.csv")
df['answer'] = df.response.apply(lambda x: x.split("Reasoning:")[0].strip())
df['reason'] = df.response.apply(lambda x: x.split("Reasoning:")[1].strip())

In [103]:
df

,post,question,response,answer,reason
0,Post: I feel like I'm stuck in a loop that I c...,Question: The answer to the question \Does the...,yes. Reasoning: The post exhibits several emot...,yes.,The post exhibits several emotional indicators...
1,"Post: I don't know why, but I feel so empty al...",Question: The answer to the question \Does the...,yes. Reasoning: The post conveys a deep sense ...,yes.,The post conveys a deep sense of emotional num...
2,Post: I keep having these thoughts that I can'...,Question: The answer to the question \Does the...,yes. Reasoning: The post exhibits strong emoti...,yes.,The post exhibits strong emotional indicators ...
3,Post: I feel like I'm watching my life pass me...,Question: The answer to the question \Does the...,yes. Reasoning: The post conveys a deep sense ...,yes.,The post conveys a deep sense of emotional num...
4,"Post: I feel so alone, even when I'm surrounde...",Question: The answer to the question \Does the...,yes. Reasoning: The post exhibits strong emoti...,yes.,The post exhibits strong emotional indicators ...
...,...,...,...,...,...
855,Post: I can't seem to get out of bed most days...,Question: The answer to the question \Does the...,yes. Reasoning: The post describes significant...,yes.,The post describes significant emotional and p...
856,"Post: I’ve been feeling really low lately, but...",Question: The answer to the question \Does the...,yes. Reasoning: The poster describes persisten...,yes.,The poster describes persistent feelings of sa...
857,Post: I’ve been having trouble sleeping for we...,Question: The answer to the question \Does the...,yes. Reasoning: Insomnia and disrupted sleep p...,yes.,"Insomnia and disrupted sleep patterns, especia..."
858,Post: I’ve been feeling really irritable and a...,Question: The answer to the question \Does the...,"yes. Reasoning: Irritability, anger, and emoti...",yes.,"Irritability, anger, and emotional outbursts c..."


In [104]:
df.drop(columns=["response"], inplace=True)

In [105]:
def tokenize_func(examples):
    prompt = [
        f"Post: {post}\nQuestion: {question}\nAnswer: {answer}\nReasoning: {reason}"
        for post, question, answer, reason in zip(
            examples["post"], examples["question"], examples["answer"], examples["reason"]
        )
    ]
    tokenized = tokenizer(prompt, padding="max_length", truncation=True, max_length=512)
    tokenized["labels"] = [
        [label if label != tokenizer.pad_token_id else -100 for label in input_ids]
        for input_ids in tokenized["input_ids"]
    ]
    return tokenized

In [106]:
from sklearn.model_selection import train_test_split
train_df, eval_df = train_test_split(df, test_size=0.2, random_state=42)
train_df = Dataset.from_pandas(train_df)
eval_df = Dataset.from_pandas(eval_df)
train_df = train_df.map(tokenize_func, batched=True)
eval_df = eval_df.map(tokenize_func, batched=True)

Map:   0%|          | 0/688 [00:00<?, ? examples/s]

Map:   0%|          | 0/172 [00:00<?, ? examples/s]

In [107]:
train_dataset = train_df.remove_columns(["post", "question", "answer", "reason", "__index_level_0__"])
eval_df_dataset = eval_df.remove_columns(["post", "question", "answer", "reason", "__index_level_0__"])

In [109]:
from transformers import TrainingArguments, Trainer
training_args = TrainingArguments(
    output_dir="./lora_mental_llama",
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    logging_dir="./logs",
    learning_rate=5e-4,
    num_train_epochs=3,
    weight_decay=0.01,
    fp16=True,
    gradient_accumulation_steps=4,
    optim="adamw_bnb_8bit",
    report_to="none"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_df_dataset
)
model.gradient_checkpointing_disable()
model.train()
trainer.train()

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,No log,0.519439
2,No log,0.529352
3,No log,0.556939


TrainOutput(global_step=258, training_loss=0.3650909837826278, metrics={'train_runtime': 480.7135, 'train_samples_per_second': 4.294, 'train_steps_per_second': 0.537, 'total_flos': 4.190787300910694e+16, 'train_loss': 0.3650909837826278, 'epoch': 3.0})

In [110]:
from google.colab import drive
drive.mount('/content/drive')
model.save_pretrained("/content/drive/MyDrive/Colab Notebook/fine_tuned_lora_mental_llama")
tokenizer.save_pretrained("/content/drive/MyDrive/Colab Notebook/fine_tuned_lora_mental_llama")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


('/content/drive/MyDrive/Colab Notebook/fine_tuned_lora_mental_llama/tokenizer_config.json',
 '/content/drive/MyDrive/Colab Notebook/fine_tuned_lora_mental_llama/special_tokens_map.json',
 '/content/drive/MyDrive/Colab Notebook/fine_tuned_lora_mental_llama/tokenizer.model',
 '/content/drive/MyDrive/Colab Notebook/fine_tuned_lora_mental_llama/added_tokens.json')